# Setting up our toy system

In [ ]:
import openpathsampling as paths
from plotting import plot
from openpathsampling.tests.test_helpers import make_1d_traj

In [ ]:
cv = paths.CoordinateFunctionCV('CV', lambda s: s.xyz[0][0])
state_A = paths.CVDefinedVolume(cv, float('-inf'), 0.0).named("A")
state_B = paths.CVDefinedVolume(cv, 1.0, float('inf')).named("B")

In [ ]:
tps_network = paths.TPSNetwork(state_A, state_B)
assert len(tps_network.sampling_ensembles) == 1
ensemble = tps_network.sampling_ensembles[0].named("TPS ensemble")

In [ ]:
scheme = paths.OneWayShootingMoveScheme(tps_network)
# NOTE: no engine given here -- currently allowed, but will cause errors if you use it!

## What does this system look like?

In [ ]:
plot([])

## A fake transition trajectory

In [ ]:
transition = make_1d_traj([i*0.1 - 0.05 for i in range(12)])

In [ ]:
plot(transition)

In [ ]:
ic = scheme.initial_conditions_from_trajectories(transition)

In [ ]:
plot(transition, ic)

# What is `init_conds`?

`initial_conditions_from_trajectories` returns a `SampleSet`. A `SampleSet` is a container for samples, which acts like a dictionary where keys can be either the replica ID for a sample or the ensemble for a sample.

In [ ]:
type(ic)

In [ ]:
len(ic)

In [ ]:
ic[0]

In [ ]:
ic[ensemble]

In [ ]:
sample = ic[0]

In [ ]:
sample.ensemble is ensemble

In [ ]:
sample.replica == 0

In [ ]:
sample.trajectory is transition

# Extracting the trajectory from the input

`initial_conditions_from_trajectories` does whatever it can to find the trajectories you need from the input you give it.

## What if the desired trajectory is a subtrajectory of the input?

In [ ]:
traj = transition + transition.reversed
ic3 = scheme.initial_conditions_from_trajectories(traj)

In [ ]:
plot(traj, ic3)

## What if the trajectory is backward?

In [ ]:
rev = transition.reversed
ic4 = scheme.initial_conditions_from_trajectories(rev)

In [ ]:
plot(rev, ic4)

## What if you give it a list of trajectories?

In [ ]:
t1 = make_1d_traj([0.25, -0.06, 0.35, -0.04])
t2 = make_1d_traj([0.2, -0.05, 0.3, 0.7, 1.05, 0.65])
ic5 = scheme.initial_conditions_from_trajectories([t1, t2])

In [ ]:
plot([t1, t2], ic5)

# What does the output message mean?

You keep seeing the text:

```
No missing ensembles.
No extra ensembles.
```

What would be missing? What would be extra?

## An example with missing ensembles

In [ ]:
interfaces = paths.VolumeInterfaceSet(cv, float("-inf"), [0.2, 0.6])
tis_network = paths.MISTISNetwork([(state_A, interfaces, state_B)])
tis_scheme = paths.DefaultScheme(tis_network)
# again, no engine, which will cause errors on dynamics

In [ ]:
plot([], interfaces=[0.2, 0.6])

In [ ]:
t1 = make_1d_traj([0.25, -0.06, 0.35, -0.04])
ic6 = tis_scheme.initial_conditions_from_trajectories(t1)

In [ ]:
plot(t1, ic6, interfaces=[0.2, 0.6])

## An example with extra ensembles

The only way to get extra ensembles is to start with an existing sample set (using the `sample_set` argument) which contains an ensemble that won't be used by the move scheme.

This would be extremely unusual in most workflows. Below, we will use the original TPS scheme with `t2`, which satisfies the scheme, but the existing sample set from the TIS scheme, which does not.

In [ ]:
t2 = make_1d_traj([0.2, -0.05, 0.3, 0.7, 1.05, 0.65])
ic7 = scheme.initial_conditions_from_trajectories(t2, sample_set=ic6)

In [ ]:
ic7 is ic6  # well, this is probably a bug.... I should revisit that

## Why no error?

You might wonder why this issue is only raised in text -- not as an error. This is because it is completely reasonable to use workflows where you build up a sample set a bit at a time.

In [ ]:
# build on an existing sample set

***Programmers***: If you want to generate an error, check out these methods:

* `scheme.assert_initial_conditions(sample_set)`: raises `AssertionError`
* `scheme.check_initial_conditions(sample_set)`: return tuple of `(missing, extra)` so you can handle it yourself

# Advanced functionality

There are many advanced approaches possible with `initial_conditions_from_trajectories`.

* change the order of trajectories to prefer longest/shortest etc.
* change whether reversed trajectories are considered
* attempt to run dynamics to generate trajectories that satisfy the ensemble

For details, check the docs on `initial_conditions_from_trajectories`.

# Future warnings

Some of the details, especially around the advanced strategies, are likely to change in future versions of OPS. It is also possible that this method will be renamed to something considerably shorter than `initial_conditions_from_trajectories`.